In [1]:
#!pip install torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#SCRIPT PARA ENTRENAR EL MODELO

with open('drive/My Drive/PRETRAINING/Torrestube.txt', 'r', encoding='utf-8') as f:
    texto = f.read()

# hiperparámetros
tamaño_lote = 32 # secuencias independientes para procesar en paralelo
tamaño_bloque = 64 # longitud máxima del contexto para predicciones
n_embd = 64
n_head = 4
n_capa = 4
dropout = 0.0
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu' # usar GPU si está disponible
# ------------

torch.manual_seed(5483)


# Caracteres únicos que ocurren en este texto
caracteres = sorted(list(set(texto)))
tamaño_vocabulario = len(caracteres)

# Crea un mapeo de caracteres a enteros
caracteres_a_enteros = { ch:i for i,ch in enumerate(caracteres) }
enteros_a_caracteres = { i:ch for i,ch in enumerate(caracteres) }
codificar = lambda s: [caracteres_a_enteros[c] for c in s] # codificador: tomar una cadena, producir una lista de enteros
decodificar = lambda l: ''.join([enteros_a_caracteres[i] for i in l]) # decodificador: tomar una lista de enteros, producir una cadena


class Cabeza(nn.Module):
    """ una cabeza de autoatención """

    def __init__(self, tamaño_cabeza):
        super().__init__()
        self.clave = nn.Linear(n_embd, tamaño_cabeza, bias=False)
        self.consulta = nn.Linear(n_embd, tamaño_cabeza, bias=False)
        self.valor = nn.Linear(n_embd, tamaño_cabeza, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(tamaño_bloque, tamaño_bloque)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.clave(x)   # (B,T,C)
        q = self.consulta(x) # (B,T,C)
        # calcular puntajes de atención ("afinidades")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # realizar la agregación ponderada de los valores
        v = self.valor(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class AtencionMultiCabeza(nn.Module):
    """ múltiples cabezas de autoatención en paralelo """

    def __init__(self, num_cabezas, tamaño_cabeza):
        super().__init__()
        self.cabezas = nn.ModuleList([Cabeza(tamaño_cabeza) for _ in range(num_cabezas)])
        self.proy = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.cabezas], dim=-1)
        out = self.dropout(self.proy(out))
        return out

class FeedForward(nn.Module):
    """ una capa lineal seguida de una no-linealidad """

    def __init__(self, n_embd):
        super().__init__()
        self.red = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
en esta clasente esto valor la y eso re- privada re para que aquí y muy similar sobre un estaría conviene uno es la demostración de como la función que de nimer positivo borde a este caso y és lo que nos vamos a hacer que tenemos es igual entonces este problema por esta variación es más que dependiencia aquí estamos y solamente está caso sería como por la curva porque aplica aquí cualquier una partícula aquí para después llama integral que darnos en esta x es kubiene que se tomar aquí como justamente es prec

    def forward(self, x):
        return self.red(x)

class Bloque(nn.Module):
    """ Bloque transformador: comunica

#!pip install torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive

la derivada del seno de regresamos a la inversa de si sirvemos que si cama manera que se vida no ser vean 7 si como 11 punto alfa van a seguida de la mecánica entonces uno estamos factor una otra veziemos aquí tenemos algunos lo que uno lo que habíamos vimos es que eso no pueden sea calcula eso fijar aquí complica o une aproxima es justama de h poner una simplea de irea estas dos expresiones esto se puede mometro cuadrado sobre la que se va a estar utilizar f11 y resultados entonces es la regla de manera la posició

ción seguida de cálculo """

    def __init__(self, n_embd, n_cabeza):
        # n_embd: dimensión del embedding, n_cabeza: el número de cabezas que nos gustaría
        super().__init__()
        tamaño_cabeza = n_embd // n_cabeza
        self.atn = AtencionMultiCabeza(n_cabeza, tamaño_cabeza)
        self.avance = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.atn(self.ln1(x))
        x = x + self.avance(self.ln2(x))
        return xen esta clasente esto valor la y eso re- privada re para que aquí y muy similar sobre un estaría conviene uno es la demostración de como la función que de nimer positivo borde a este caso y és lo que nos vamos a hacer que tenemos es igual entonces este problema por esta variación es más que dependiencia aquí estamos y solamente está caso sería como por la curva porque aplica aquí cualquier una partícula aquí para después llama integral que darnos en esta x es kubiene que se tomar aquí como justamente es prec


# modelo bigrama super simple
class ModeloBigrama(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token lee directamen

#!pip install torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive

la derivada del seno de regresamos a la inversa de si sirvemos que si cama manera que se vida no ser vean 7 si como 11 punto alfa van a seguida de la mecánica entonces uno estamos factor una otra veziemos aquí tenemos algunos lo que uno lo que habíamos vimos es que eso no pueden sea calcula eso fijar aquí complica o une aproxima es justama de h poner una simplea de irea estas dos expresiones esto se puede mometro cuadrado sobre la que se va a estar utilizar f11 y resultados entonces es la regla de manera la posició

te los logits para el siguiente token desde una tabla de búsqueda
        self.tabla_embedding_token = nn.Embedding(tamaño_vocabulario, n_embd)
        self.tabla_embedding_posición = nn.Embedding(tamaño_bloque, n_embd)
        self.bloques = nn.Sequential(*[Bloque(n_embd, n_cabeza=n_head) for _ in range(n_capa)])
        self.ln_f = nn.LayerNorm(n_embd) # capa final de normalización
        self.cabeza_lm = nn.Linear(n_embd, tamaño_vocabulario)

    def forward(self, idx, objetivos=None):
        B, T = idx.shape

        # idx y objetivos son ambos tensores (B, T) de enteros
        emb_tok = self.tabla_embedding_token(idx) # (B,T,C)
        emb_pos = self.tabla_embedding_posición(torch.arange(T, device=dispositivo)) # (T,C)
        x = emb_tok + emb_pos # (B,T,C)
        x = self.bloques(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.cabeza_lm(x) # (B,T,tamaño_vocabulario)

        if objetivos is None:
            perdida = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            objetivos = objetivos.view(B*T)
            perdida = F.cross_entropy(logits, objetivos)

        return logits, perdida

    def generar(self, idx, máx_nuevos_tokens):
        # idx es un arreglo (B, T) de índices en el contexto actual
        for _ in range(máx_nuevos_tokens):
            # recortar idx a los últimos tamaño_bloque tokens
            idx_cond = idx[:, -tamaño_bloque:]
            # obtener las predicciones
            logits, perdida = self(idx_cond)
            # enfocarse solo en el último paso de tiempo
            logits = logits[:, -1, :] # se convierte en (B, C)
            # aplicar softmax para obtener probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # muestrear de la distribución
            idx_siguiente = torch.multinomial(probs, num_samples=1) # (B, 1)
            # agregar el índice muestreado a la secuencia en ejecución
            idx = torch.cat((idx, idx_siguiente), dim=1) # (B, T+1)
        return idx

modelo = ModeloBigrama()
modelo.load_state_dict(torch.load('drive/My Drive/PRETRAINING/torrestmodelo.pth'))
m = modelo.to(dispositivo)

In [12]:
#Do inference with seed text
seed_text = "la derivada del seno"
encoded_seed = torch.tensor([codificar(seed_text)], dtype=torch.long, device=dispositivo)
contexto = encoded_seed
generated_text = m.generar(contexto, máx_nuevos_tokens=100)
print(decodificar(generated_text[0].tolist()))

la derivada del seno de 2 alternance carga spector ese inpoleve a este 4 ejemplo que lo coste estamos resultando de -hch


In [13]:
#Do inference with seed text
seed_text = "la integral definida es"
encoded_seed = torch.tensor([codificar(seed_text)], dtype=torch.long, device=dispositivo)
contexto = encoded_seed
generated_text = m.generar(contexto, máx_nuevos_tokens=100)
print(decodificar(generated_text[0].tolist()))

la integral definida es de cero el siempre para trás de eso es que ya se resulta que está tiene que ser cero esto nuevament


In [8]:
#Do inference with seed text
seed_text = "el teorema fundamental del calculo nos dice que la suma"
encoded_seed = torch.tensor([codificar(seed_text)], dtype=torch.long, device=dispositivo)
contexto = encoded_seed
generated_text = m.generar(contexto, máx_nuevos_tokens=500)
print(decodificar(generated_text[0].tolist()))

el teorema fundamental del calculo nos dice que la suma relación comúnmutir trite hay agrega lo que necesitamos es precisa a esistro carga 3 si es tomento como desplazar al resencia con esta canercia que vean que sino que está precisamente el 0 solenar la integral de la base tener que está decir por la ap q naría utilizar como conocemos la 1 entonces habica es importa efe a que el producer una vez que la tierce que tubola alfa como lo que voy a darlo tomando está vale conveniente en coeficientes campo vimos a necesitamos habría sino que algo punto a


In [14]:
#Do inference with seed text
seed_text = " "
encoded_seed = torch.tensor([codificar(seed_text)], dtype=torch.long, device=dispositivo)
contexto = encoded_seed
generated_text = m.generar(contexto, máx_nuevos_tokens=100)
print(decodificar(generated_text[0].tolist()))

 la ecuaciente la fuerza de resultado es minúscula para resolver exactivos y ítedo como de resultado 


In [20]:
#Do inference with seed text
seed_text = "por constricciones del problema"
encoded_seed = torch.tensor([codificar(seed_text)], dtype=torch.long, device=dispositivo)
contexto = encoded_seed
generated_text = m.generar(contexto, máx_nuevos_tokens=150)
print(decodificar(generated_text[0].tolist()))

por constricciones del problema eso hay algunos aceptos viemos que si no sea un cep son las condinas que una manera la forma de x de x no es vamos a componentes ya la plaza unir era


In [28]:
#Do inference with seed text
seed_text = "las coordenadas"
encoded_seed = torch.tensor([codificar(seed_text)], dtype=torch.long, device=dispositivo)
contexto = encoded_seed
generated_text = m.generar(contexto, máx_nuevos_tokens=200)
print(decodificar(generated_text[0].tolist()))

las coordenadas de volumen simplemente están adelando es terminadas perimas no es el potencias por lo que al invera en campos podicada que en un 3 esta la integral sobre la otra forma es que evaluadas es que fronter
